#### https://online.stat.psu.edu/stat501/lesson/10

#### Most often than not, a researcher has a large set of candidate predictor variabels from which he tries to identify the most appropriate predictors to include in his regression model

#### Two variable selection methods: stepwise regression and best subsets regression

In [1]:
import os
import statsmodels.api as sm
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.display import Image
from statsmodels.formula.api import ols
import scipy.stats
from matplotlib import pyplot as plt
filedir = 'STAT501_Lesson10'
os.listdir(filedir)

['.DS_Store',
 'Physical.txt',
 'iqsize.txt',
 'bloodpress.txt',
 'peru.txt',
 'cement.txt',
 'martian.txt']

If a regression model is correctly specificied:
<ol>
    <li>unbiased regression coefficients</li>
    <li>unbiased prediction of y</li>
    <li>unbiased estimate of the error variance / MSE</li>
</ol>

If a regression model is underspecificied:
<ol>
    <li>biased regression coefficients</li>
    <li>biased prediction of y</li>
    <li>overestimate of the error variance / MSE, yielding wider confidence intervals</li>
</ol>

If a regression model is overspecificied:
<ol>
    <li>unbiased regression coefficients</li>
    <li>unbiased prediction of y</li>
    <li>unbiased estimate of the error variance; but we have more paramters than needed, so MSE has fewer degrees of freedom, MSE larger, yielding wider confidence intervals</li>
</ol>

#### 1. Stepwise Regression

In [3]:
df = pd.read_table(os.path.join(filedir, 'iqsize.txt'), encoding="utf-8")
df

,PIQ,Brain,Height,Weight
0,124,81.69,64.5,118
1,150,103.84,73.3,143
2,128,96.54,68.8,172
3,134,95.15,65.0,147
4,110,92.88,69.0,146
5,131,99.13,64.5,138
6,98,85.43,66.0,175
7,84,90.49,66.3,134
8,147,95.55,68.8,172
9,124,83.39,64.5,118


In [5]:
candidate_predictors = ['Brain','Height','Weight']
y = df['PIQ']
X = df[candidate_predictors]
X = sm.add_constant(X)

def get_p(y, X, features_used):
    model = sm.OLS(y,X[['const']+features_used]).fit()
    return model.pvalues

get_p(y,X,['Brain'])

/Users/yuehuazhuang/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


const    0.915838
Brain    0.019354
dtype: float64

In [10]:
get_p(y,X,['Height'])

const     0.027942
Height    0.578020
dtype: float64

In [6]:
get_p(y,X,['Weight'])

const     0.000063
Weight    0.988057
dtype: float64

#### pick Brain

In [7]:
get_p(y,X,['Brain','Height'])

const     0.054243
Brain     0.000604
Height    0.009399
dtype: float64

In [8]:
get_p(y,X,['Brain','Weight'])

const     0.912686
Brain     0.006592
Weight    0.150705
dtype: float64

#### pick Brain, Height

In [11]:
get_p(y,X,['Brain','Height','Weight'])

const     0.085979
Brain     0.000856
Height    0.033034
Weight    0.997750
dtype: float64

Stop with Brain, Height

https://towardsdatascience.com/5-feature-selection-method-from-scikit-learn-you-should-know-ed4d116e4172

https://scikit-learn.org/stable/modules/feature_selection.html

#### 2. Best Subsets Regression

In [32]:
Image(url=os.path.join(filedir, "best_subsets_result.png"), width=600, height=600)

In [20]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_regression, f_classif

In [15]:
df = pd.read_table(os.path.join(filedir, 'cement.txt'), encoding="utf-8")
df

,y,x1,x2,x3,x4
0,78.5,7,26,6,60
1,74.3,1,29,15,52
2,104.3,11,56,8,20
3,87.6,11,31,8,47
4,95.9,7,52,6,33
5,109.2,11,55,9,22
6,102.7,3,71,17,6
7,72.5,1,31,22,44
8,93.1,2,54,18,22
9,115.9,21,47,4,26


In [25]:
X = df[['x1','x2','x3','x4']]
y = df['y']
select = SelectKBest(score_func=f_classif, k=3)
z = select.fit_transform(X, y)
select.get_support()

/Users/yuehuazhuang/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:111: RuntimeWarning: invalid value encountered in true_divide
  msw = sswn / float(dfwn)


array([False,  True,  True,  True])

In [29]:
X = df[['x1','x2','x3','x4']]
y = df['y']
select = SelectKBest(score_func=f_regression, k=3)
z = select.fit_transform(X, y)
select.get_support()

array([ True,  True, False,  True])

#### 3. Information Criteria and PRESS

#### 4. Cross Validation